In [675]:
import bs4
from bs4 import BeautifulSoup
import csv
import requests
import time
import pandas as pd
import urllib
import re
import pickle
import numpy as np
import time

In [676]:
url = 'https://www.imdb.com/search/title/?release_date=2017&sort=num_votes,desc&page=2&ref_=adv_nxt'
request = requests.request('GET', url, headers = {"Accept-Language": "en-US, en;q=0.5"})

In [384]:
soup =  BeautifulSoup(request.text, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [385]:
movies_data_container = soup.find_all('div', attrs = {'class': 'lister-item mode-advanced'})

In [403]:
movie_data_container[49].h3.a.text

'American Made'

In [387]:
movies_data_container[0].h3.find('span', attrs = {'class': 'lister-item-year text-muted unbold'}).text.strip('()')
#movie_data_container[0].h3.find('span', class_ = 'lister-item-year text-muted unbold').text

'2017'

In [73]:
movies_data_container[0].strong.text

'8.1'

In [374]:
movie_data_container[0].find('div', attrs = {'class': 'ratings-metascore'}).text[1:3]


'77'

In [375]:
movie_data_container[0].find('span', attrs = {'name': 'nv'}).text.replace(',', '')

'671393'

In [408]:
movie_name = []
yor = []
IMDB_r = []
metascore = []
total_votes = []

for i in range(len(movies_data_container)):
    if movie_data_container[i].find('div', attrs = {'class': 'ratings-metascore'}) != None:
        movie_name.append(movies_data_container[i].h3.a.text)
        yor.append((movies_data_container[i].h3.find('span', attrs = {'class': 'lister-item-year text-muted unbold'}).text.strip('()')))
        IMDB_r.append(float(movies_data_container[i].strong.text))
        metascore.append((movie_data_container[i].find('div', attrs = {'class': 'ratings-metascore'})).text[1:3])
        total_votes.append(int(movies_data_container[i].find('span', attrs = {'name': 'nv'}).text.replace(',','')))
    else:
        movie_name.append(movies_data_container[i].h3.a.text)
        yor.append((movies_data_container[i].h3.find('span', attrs = {'class': 'lister-item-year text-muted unbold'}).text.strip('()')))
        IMDB_r.append(float(movies_data_container[i].strong.text))
        metascore.append(None)
        total_votes.append(int(movies_data_container[i].find('span', attrs = {'name': 'nv'}).text.replace(',','')))

In [418]:
df = pd.DataFrame(zip(movie_name, yor, IMDB_r, total_votes, metascore), columns = ['Name', 'Year', 'IMDB score', 'Votes', 'Metascore'])
df.head()

,Name,Year,IMDB score,Votes,Metascore
0,Logan,2017,8.1,671396,77
1,Thor: Ragnarok,2017,7.9,621834,74
2,Wonder Woman,2017,7.4,597480,76
3,Guardians of the Galaxy Vol. 2,2017,7.6,593007,67
4,Dunkirk,2017,7.8,579134,94


##  2010 - 2020 data  -  Top 1000 each year

In [746]:
years = np.arange(2010, 2021, 1)
title_counts = np.arange(1, 1001, 50)

In [747]:
from IPython.core.display import clear_output
req = 0
movie_name = []
yor = []
IMDB_r = []
metascore = []
total_votes = []
for year in years:
    
    time.sleep(np.random.randint(5, 18))
    for tc in title_counts:
        time.sleep(np.random.rand())
        url = f'https://www.imdb.com/search/title/?release_date={year}-01-01,{year}-12-31&sort=num_votes,desc&start={tc}&ref_=adv_prv'
        start = time.time()
        request = requests.request('GET', url, headers = {"Accept-Language": "en-US, en;q=0.5"})
        end = time.time()
        soup =  BeautifulSoup(request.text, 'html.parser')
        movies_data_container = soup.find_all('div', attrs = {'class': 'lister-item mode-advanced'})

        for i in range(len(movies_data_container)):
            if movie_data_container[i].find('div', attrs = {'class': 'ratings-metascore'}) != None:
                movie_name.append(movies_data_container[i].h3.a.text)
                yor.append((movies_data_container[i].h3.find('span', attrs = {'class': 'lister-item-year text-muted unbold'}).text.strip('()')))
                IMDB_r.append(float(movies_data_container[i].strong.text))
                metascore.append((movie_data_container[i].find('div', attrs = {'class': 'ratings-metascore'})).text[1:3])
                total_votes.append(int(movies_data_container[i].find('span', attrs = {'name': 'nv'}).text.replace(',','')))
            else:
                movie_name.append(movies_data_container[i].h3.a.text)
                yor.append((movies_data_container[i].h3.find('span', attrs = {'class': 'lister-item-year text-muted unbold'}).text.strip('()')))
                IMDB_r.append(float(movies_data_container[i].strong.text))
                metascore.append(None)
                total_votes.append(int(movies_data_container[i].find('span', attrs = {'name': 'nv'}).text.replace(',','')))
        req += 1
        print(f'Request: {req}; Frequency: {end-start} requests/s')
        clear_output(wait = True)

Request: 220; Frequency: 2.521258592605591 requests/s


In [758]:
df = pd.DataFrame(zip(movie_name, yor, IMDB_r, total_votes, metascore), columns = ['Name', 'Year', 'IMDB score', 'Votes', 'Metascore'])
df

,Name,Year,IMDB score,Votes,Metascore
0,Inception,2010,8.8,2128303,77
1,Shutter Island,2010,8.2,1169467,74
2,The Walking Dead,2010–2022,8.2,877009,76
3,Sherlock,2010–2017,9.1,832609,67
4,Toy Story 3,2010,8.2,773734,94
...,...,...,...,...,...
10995,The Windermere Children,2020,7.2,2546,29
10996,Altered Carbon,2018–2020,6.4,2544,37
10997,Mr. Queen,2020–,8.9,2540,51
10998,Hightown,2020–,6.5,2535,None


In [759]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        11000 non-null  object 
 1   Year        11000 non-null  object 
 2   IMDB score  11000 non-null  float64
 3   Votes       11000 non-null  int64  
 4   Metascore   9240 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 300.8+ KB


In [760]:
df.loc[df['IMDB score'] == df['IMDB score'].max()]

,Name,Year,IMDB score,Votes,Metascore
3084,Breaking Bad,2008–2013,10.0,139268,None


In [980]:
import bs4
from bs4 import BeautifulSoup
import csv
import requests
import time
import pandas as pd
import urllib
import re
import pickle
import datetime
import numpy as np

In [992]:
from IPython.core.display import clear_output
Average_temperature = []; Average_humidity = []; Average_dewpoint = []; Average_barometer = []; Average_windspeed = []; Average_gustspeed = []; Average_direction = []; Rainfall_for_month = []; 
Rainfall_for_year = []; Maximum_rain_per_minute = []; Maximum_temperature = []; Minimum_temperature = []; Maximum_humidity = []; Minimum_humidity = []; Maximum_pressure = []; Minimum_pressure = []; 
Maximum_windspeed = []; Maximum_gust_speed = []; Maximum_heat_index = []
req = 0; rtables = []; date = []

start_time = pd.to_datetime('2009-01-01')
end_time = pd.to_datetime('2009-10-28')
#print(start_time.date())
#print(end_time.date())

for year in range(start_time.year, end_time.year+1, 1):
    for month in ['{}'.format(x) for x in range(start_time.month, 4, 1)]:
        if len(month) < 2:
            month = '0'+month
            url = 'http://www.estesparkweather.net/archive_reports.php?date={}{}'.format(year, month)
        else:
            month = month
            url = 'http://www.estesparkweather.net/archive_reports.php?date={}{}'.format(year, month)
            
        #time.sleep(np.random.randint(2, 6))
        
        start = time.time()
        request = requests.request('GET', url, headers = {"Accept-Language": "en-US"})
        end = time.time()
        
        soup = BeautifulSoup(request.text, 'html.parser')
        raw_tables = soup.find_all('table')

        for raw_table in raw_tables:
            #  taking only useful contents
            ls_raw_table = raw_table.find_all('td')[0]
            if 'for Month of' in (ls_raw_table.text):
                break
            else:
                rtables.append(raw_table)

        for rtable in rtables:
            variables_raw = []
            data = []
            raw_date = ''
            table = (rtable.find_all('td'))
            #print(table)
            if len(raw_date) == 0:
                    raw_date = (table[0].text)
                    raw_date = (raw_date.split(' '))
                    rdate = "{}/{}/{}".format(raw_date[0], raw_date[1], year)
                    if rdate not in ['Feb/29/2017', 'Feb/29/2018']:
                        date_ = pd.to_datetime(rdate).strftime('%Y-%m-%d')
                        date += [date_]
            
            for i in range(1, len(table)):
                break
                
            
            
            
            
            for i in range(1, len(table)):
                break
                if len(raw_date) == 0:
                    raw_date = (table[0].text)
                    raw_date = (raw_date.split(' '))
                    rdate = "{}/{}/{}".format(raw_date[0], raw_date[1], year)
                    if rdate not in ['Feb/29/2017', 'Feb/29/2018']:
                        date_ = pd.to_datetime(rdate).strftime('%Y-%m-%d')
                        date += [date_]
                        
                
                if rdate not in ['Feb/29/2017', 'Feb/29/2018']:
                    if i % 2 == 0:
                        if '.' in table[i].text and table[i].text not in ['', ' ']:
                            match = re.search(r"\s\d*.\d*", table[i].text)
                            value = match.group().lstrip()
                            data.append(float(value))
                        else:
                            match = re.search(r"\s\d*", table[i].text)
                            value = match.group().lstrip()
                            if value not in ['', ' ']:
                                data.append(int(value))
                            else:
                                data.append(0)
                    else:
                        variables_raw.append('_'.join(table[i].text.split()))

            for v, d in zip(variables_raw, data):
                break
                if v == 'Average_temperature':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Average_humidity':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Average_dewpoint':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Average_barometer':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Average_windspeed':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Average_gustspeed':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Average_direction':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Rainfall_for_month':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Rainfall_for_year':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Maximum_rain_per_minute':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Maximum_temperature':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Minimum_temperature':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Maximum_humidity':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Minimum_humidity':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Maximum_pressure':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Minimum_pressure':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Maximum_windspeed':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Maximum_gust_speed':
                    exec("{}.append({})".format(v, 'd'))
                if v == 'Maximum_heat_index':
                    exec("{}.append({})".format(v, 'd'))
        #print(len(date))
        req += 1
        print('Request: {}; Response Time : {} s'.format(req, end-start))
        print('{}-{}'.format(month, year))
        print(url)
        clear_output(wait = True)

Request: 3; Response Time : 1.7954003810882568 s
03-2009
http://www.estesparkweather.net/archive_reports.php?date=200903


In [989]:
print(len(Average_temperature))
x = [print(x) for x in date]

177
2009-01-01
2009-01-02
2009-01-03
2009-01-04
2009-01-05
2009-01-06
2009-01-07
2009-01-08
2009-01-09
2009-01-10
2009-01-11
2009-01-12
2009-01-13
2009-01-14
2009-01-15
2009-01-16
2009-01-17
2009-01-18
2009-01-19
2009-01-20
2009-01-21
2009-01-22
2009-01-23
2009-01-24
2009-01-25
2009-01-26
2009-01-27
2009-01-28
2009-01-29
2009-01-30
2009-01-31
2009-01-01
2009-01-02
2009-01-03
2009-01-04
2009-01-05
2009-01-06
2009-01-07
2009-01-08
2009-01-09
2009-01-10
2009-01-11
2009-01-12
2009-01-13
2009-01-14
2009-01-15
2009-01-16
2009-01-17
2009-01-18
2009-01-19
2009-01-20
2009-01-21
2009-01-22
2009-01-23
2009-01-24
2009-01-25
2009-01-26
2009-01-27
2009-01-28
2009-01-29
2009-01-30
2009-01-31
2009-02-01
2009-02-02
2009-02-03
2009-02-04
2009-02-05
2009-02-06
2009-02-07
2009-02-08
2009-02-09
2009-02-10
2009-02-11
2009-02-12
2009-02-13
2009-02-14
2009-02-15
2009-02-16
2009-02-17
2009-02-18
2009-02-19
2009-02-20
2009-02-21
2009-02-22
2009-02-23
2009-02-24
2009-02-25
2009-02-26
2009-02-27
2009-02-28
2009-0

In [991]:
table

[<td colspan="2">Mar 31 Average and Extremes</td>,
 <td>Average temperature</td>,
 <td> 25.8 °F</td>,
 <td>Average humidity</td>,
 <td> 40 %</td>,
 <td>Average dewpoint</td>,
 <td> 4.5 °F</td>,
 <td>Average barometer</td>,
 <td> 29.8 in.</td>,
 <td>Average windspeed</td>,
 <td> 16.1 mph</td>,
 <td>Average gustspeed</td>,
 <td> 23.2 mph</td>,
 <td>Average direction</td>,
 <td> 266 ° ( W )</td>,
 <td>Rainfall for month</td>,
 <td> 0.40 in.</td>,
 <td>Rainfall for year</td>,
 <td> 0.64 in.</td>,
 <td>Maximum rain per minute</td>,
 <td> 0.00 in. on day 31 at time 23:53</td>,
 <td>Maximum temperature</td>,
 <td> 33.4 °F on day 31 at time 15:05</td>,
 <td>Minimum temperature</td>,
 <td> 17.6 °F on day 30 at time 01:03</td>,
 <td>Maximum humidity</td>,
 <td> 69 % on day 31 at time 23:53</td>,
 <td>Minimum humidity</td>,
 <td> 27 % on day 31 at time 15:18</td>,
 <td>Maximum pressure</td>,
 <td> 29.977  in. on day 31 at time 02:30</td>,
 <td>Minimum pressure</td>,
 <td> 29.661  in. on day 31 at